# Inspecting Satellite Imagery using Rasterio
## A first look at satellite data with Python

At this point, you will have learned different ways of searching for, filtering, and downloading satellite imagery. Now let's use one of these acquired datasets and dig into it a bit with Python.

Here we're going to use a Python library called `rasterio`: you may be familiar with it already, or perhaps with the related C library, `GDAL`. If you've used `numpy` before, working with `rasterio` will feel very familiar.

In [1]:
import rasterio

# feel free to replace 'example' with your own GeoTIFF:
# note that this Notebook will assume we're working with PlanetScope 4-band imagery
image_file = "example.tif"

satdat = rasterio.open(image_file)

## Basic details
What can we learn about this satellite image using just Python?

In [2]:
# Get the bounding box of this GeoTIFF

satdat.bounds

BoundingBox(left=544170.0, bottom=3759147.0, right=550146.0, top=3766416.0)

In [3]:
# Get dimensions, in map units (using the example GeoTIFF, that's meters)

width = satdat.bounds.right - satdat.bounds.left
height = satdat.bounds.top - satdat.bounds.bottom

print("Width: {}, Height: {}".format(width, height))

Width: 5976.0, Height: 7269.0


In [4]:
# Get dimensions, in pixels

px_width = satdat.width
px_height = satdat.height
print("Width: {}, Height: {}".format(px_width, px_height))

Width: 1992, Height: 2423


In [5]:
# How many meters to a pixel?

w = width / px_width
h = height / px_height

w, h

(3.0, 3.0)

In [6]:
# Get coordinate reference system

satdat.crs

CRS({'init': 'epsg:32611'})

In [7]:
# Get coordinates of top-left & bottom right corners

# NOTE: how to do this depends on your Rasterio version --
# if you're running against a pre-1.0 release use satdat.affine instead

topleft = satdat.transform * (0, 0)
botright = satdat.transform * (width, height)
    
print("Top left corner coordinates: {}".format(topleft))
print("Bottom right corner coordinates: {}".format(botright))

Top left corner coordinates: (544170.0, 3766416.0)
Bottom right corner coordinates: (562098.0, 3744609.0)


In [8]:
# Another way of viewing most of the previous values:
# Get the basic metadata of this GeoTIFF

satdat.meta

{'count': 4,
 'crs': CRS({'init': 'epsg:32611'}),
 'driver': 'GTiff',
 'dtype': 'uint16',
 'height': 2423,
 'nodata': 0.0,
 'transform': Affine(3.0, 0.0, 544170.0,
       0.0, -3.0, 3766416.0),
 'width': 1992}

## Bands
So far, we haven't done too much geospatial-raster-specific work yet. Since we know we're inspecting a multispectral satellite image, let's see what we can learn about its bands.

In [9]:
# Get the number of bands by listing their indices

satdat.indexes

(1, 2, 3, 4)

Because we know we're look at a PlanetScope 4-band analytic satellite image, we can define the bands by their order:

In [10]:
# PlanetScope 4-band band order: BGRN

blue = satdat.read(1)
green = satdat.read(2)
red = satdat.read(3)
nir = satdat.read(4)

# or:
# blue, green, red, nir = satdat.read()

## Pixels

In a raster dataset, each pixel has a value. Pixels are arranged in a grid, and pixels representing equivalent data have the same value:

![pixels2.png](pixels2.png)

In [12]:
# bands are stored as numpy arrays

print(type(blue))

<class 'numpy.ndarray'>


In [13]:
# how many dimensions would a raster have?

print(blue.ndim)

2


In [14]:
# take a look at the summarized array

print(blue)

[[7261 7137 7087 ... 7015 6868 6891]
 [7180 7076 7166 ... 7387 7391 7431]
 [7424 7436 7443 ... 7497 7713 7760]
 ...
 [7791 7840 8139 ... 7108 7086 7267]
 [7873 8132 8441 ... 7134 7023 7042]
 [8320 8464 8542 ... 6893 6921 6989]]


In [15]:
# Output a min & max pixel value in each band

# we'll need to use numpy directly for this
import numpy

print(numpy.amin(blue), numpy.amax(blue))
print(numpy.amin(green), numpy.amax(green))
print(numpy.amin(red), numpy.amax(red))
print(numpy.amin(nir), numpy.amax(nir))

3493 22058
2905 17846
2048 19242
1132 13004


In [16]:
# Let's grab the pixel 2km east and 2km south of the upper left corner

# get the pixel 
px_x = satdat.bounds.left + 2000
px_y = satdat.bounds.top - 2000

row, col = satdat.index(px_x, px_y)

# Now let's look at the value of each band at this pixel
print("Red: {}".format(red[row, col]))
print("Green: {}".format(green[row, col]))
print("Blue: {}".format(blue[row, col]))
print("NIR: {}".format(nir[row, col]))

Red: 8296
Green: 8579
Blue: 8642
NIR: 7606
